This takes way too long. Bigger buffer?

In [1]:
import json
import sqlite3

conn = sqlite3.Connection('data/data.db')
cur = conn.cursor()

create_embeddings = """
    CREATE TABLE embeddings (
        id INTEGER PRIMARY KEY,         
        vector TEXT              
    );
"""
cur.execute(create_embeddings)
conn.commit()

insert_embeddings = """
    INSERT INTO embeddings (id, vector) VALUES (?, ?);
"""

buffer = []
with open('data/embeddings.jsonl', 'rt') as file:
    for line in file:
        j = json.loads(line)
        buffer.append((
            j['id'], 
            json.dumps(j['embedding'])
        ))
        if len(buffer) > 10000:
            print("Writing buffer.")
            cur.executemany(
                insert_embeddings, 
                buffer
            )
            conn.commit()
            buffer = []

if len(buffer) > 0:
    print("Writing buffer.")        
    cur.executemany(
        insert_embeddings, 
        buffer
    )
    conn.commit()
    buffer = []

Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing 

In [2]:
import json
import sqlite3
import pandas as pd

FROM_DIR = 'source'

conn = sqlite3.Connection('data/data.db')
cur = conn.cursor()

create_articles = """
    CREATE TABLE articles (
        id INTEGER PRIMARY KEY,         
        url TEXT,
        title TEXT,
        clean_title TEXT,
        count INTEGER
    );
"""
cur.execute(create_articles)
conn.commit()

insert_articles = """
    INSERT INTO articles (id, url, title, clean_title, count) VALUES (?, ?, ?, ?, ?);
"""

buffer = []
with open('../transformed/data/selected_articles.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        j = json.loads(line)
        buffer.append(
            (j['id'], j['url'], j['title'], j['title'].lower().strip(), j['count'])
        )
        if len(buffer) > 10000:
            print("Writing buffer.")
            cur.executemany(
                insert_articles,
                buffer
            )
            conn.commit()
            buffer = []

    if len(buffer) > 0:
        print("Writing buffer.")
        cur.executemany(
            insert_articles,
            buffer
        )
        conn.commit()
        buffer = []

Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing 

In [3]:
import json
import sqlite3
import os 
import pandas as pd

conn = sqlite3.Connection('data/data.db')
cur = conn.cursor()

index1 = """
    CREATE INDEX articles_title_index ON articles(title);
"""

index2 = """
    CREATE INDEX articles_clean_title_index ON articles(clean_title);
"""

index3 = """
    CREATE INDEX articles_url_index ON articles(url);
"""

index4 = """
    CREATE INDEX articles_count_index ON articles(count);
"""

index5 = """
    CREATE INDEX idx_articles_clean_title_count ON articles(clean_title, count DESC);
"""

cur.execute(index1)
cur.execute(index2)
cur.execute(index3)
cur.execute(index4)
cur.execute(index5)

conn.commit()
